In [1]:
from sqlalchemy import (
  create_engine, Column, Integer, String, Boolean, ForeignKey, func, distinct
)
from sqlalchemy.orm import declarative_base, aliased, sessionmaker

In [2]:
Base = declarative_base()

class App(Base):
    __tablename__ = 'app'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    def __repr__(self):
       return f"<App (name='{self.name}')>" 

class Sdk(Base):
    __tablename__ = 'sdk'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    slug = Column(String)
    def __repr__(self):
       return f"<SDK (name='{self.name}' slug='{self.slug}')>" 

class AppSdk(Base):
    __tablename__ = 'app_sdk'
    app_id = Column(Integer, ForeignKey("app.id"), primary_key=True)
    sdk_id = Column(Integer, ForeignKey("sdk.id"), primary_key=True)
    installed = Column(Boolean)
    
    def __repr__(self):
       return f"<app_id='{self.app_id}' self.sdk='{self.sdk_id}' installed={self.installed}>" 

In [3]:
engine = create_engine("sqlite:///data.db", echo=True, future=True)
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
sdk_choice = "paypal braintree stripe".split() # slugs

sdk_sql_selector = Sdk.slug.in_(sdk_choice)

In [5]:
T1 = aliased(AppSdk)
T2 = aliased(AppSdk)

Sdk1 = aliased(Sdk)
Sdk2 = aliased(Sdk)

root_query = (
  session
  .query(T1, T2, Sdk1, Sdk2, func.count(distinct(T1.app_id)))
  .filter(
    T1.app_id == T2.app_id,
    T1.sdk_id==Sdk1.id,
    T2.sdk_id==Sdk2.id,
    T1.installed == False,
    T2.installed == True,
  )
)


churn_main = (
  root_query
  .filter(
    Sdk1.slug.in_(sdk_choice),
    Sdk2.slug.in_(sdk_choice)
   )
  .group_by(Sdk1.id, Sdk2.id)
).all()

print()
for _, _, sdk1, sdk2, count in churn_main:
  print(sdk1.slug, sdk2.slug, count)

2022-09-11 20:00:18,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-11 20:00:18,722 INFO sqlalchemy.engine.Engine SELECT app_sdk_1.app_id AS app_sdk_1_app_id, app_sdk_1.sdk_id AS app_sdk_1_sdk_id, app_sdk_1.installed AS app_sdk_1_installed, app_sdk_2.app_id AS app_sdk_2_app_id, app_sdk_2.sdk_id AS app_sdk_2_sdk_id, app_sdk_2.installed AS app_sdk_2_installed, sdk_1.id AS sdk_1_id, sdk_1.name AS sdk_1_name, sdk_1.slug AS sdk_1_slug, sdk_2.id AS sdk_2_id, sdk_2.name AS sdk_2_name, sdk_2.slug AS sdk_2_slug, count(DISTINCT app_sdk_1.app_id) AS count_1 
FROM app_sdk AS app_sdk_1, app_sdk AS app_sdk_2, sdk AS sdk_1, sdk AS sdk_2 
WHERE app_sdk_1.app_id = app_sdk_2.app_id AND app_sdk_1.sdk_id = sdk_1.id AND app_sdk_2.sdk_id = sdk_2.id AND app_sdk_1.installed = 0 AND app_sdk_2.installed = 1 AND sdk_1.slug IN (?, ?, ?) AND sdk_2.slug IN (?, ?, ?) GROUP BY sdk_1.id, sdk_2.id
2022-09-11 20:00:18,723 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ('paypal', 'braintree', 'stripe', 

In [6]:
churn_from_none = (
  root_query
  .filter(
    Sdk1.slug.not_in(sdk_choice),
    Sdk2.slug.in_(sdk_choice)
   )
  .group_by(Sdk2.id)
).all()

print()
for _, _, _, sdk2, count in churn_from_none:
  print(sdk2.slug, count)

2022-09-11 20:00:18,745 INFO sqlalchemy.engine.Engine SELECT app_sdk_1.app_id AS app_sdk_1_app_id, app_sdk_1.sdk_id AS app_sdk_1_sdk_id, app_sdk_1.installed AS app_sdk_1_installed, app_sdk_2.app_id AS app_sdk_2_app_id, app_sdk_2.sdk_id AS app_sdk_2_sdk_id, app_sdk_2.installed AS app_sdk_2_installed, sdk_1.id AS sdk_1_id, sdk_1.name AS sdk_1_name, sdk_1.slug AS sdk_1_slug, sdk_2.id AS sdk_2_id, sdk_2.name AS sdk_2_name, sdk_2.slug AS sdk_2_slug, count(DISTINCT app_sdk_1.app_id) AS count_1 
FROM app_sdk AS app_sdk_1, app_sdk AS app_sdk_2, sdk AS sdk_1, sdk AS sdk_2 
WHERE app_sdk_1.app_id = app_sdk_2.app_id AND app_sdk_1.sdk_id = sdk_1.id AND app_sdk_2.sdk_id = sdk_2.id AND app_sdk_1.installed = 0 AND app_sdk_2.installed = 1 AND (sdk_1.slug NOT IN (?, ?, ?)) AND sdk_2.slug IN (?, ?, ?) GROUP BY sdk_2.id
2022-09-11 20:00:18,750 INFO sqlalchemy.engine.Engine [generated in 0.00531s] ('paypal', 'braintree', 'stripe', 'paypal', 'braintree', 'stripe')

paypal 47
stripe 121
braintree 80


In [7]:
churn_to_none = (
  root_query
  .filter(
    Sdk1.slug.in_(sdk_choice),
    Sdk2.slug.not_in(sdk_choice)
   )
  .group_by(Sdk1.id)
).all()

print()
for _, _, sdk1, _, count in churn_to_none:
  print(sdk1.slug, count)

2022-09-11 20:00:18,792 INFO sqlalchemy.engine.Engine SELECT app_sdk_1.app_id AS app_sdk_1_app_id, app_sdk_1.sdk_id AS app_sdk_1_sdk_id, app_sdk_1.installed AS app_sdk_1_installed, app_sdk_2.app_id AS app_sdk_2_app_id, app_sdk_2.sdk_id AS app_sdk_2_sdk_id, app_sdk_2.installed AS app_sdk_2_installed, sdk_1.id AS sdk_1_id, sdk_1.name AS sdk_1_name, sdk_1.slug AS sdk_1_slug, sdk_2.id AS sdk_2_id, sdk_2.name AS sdk_2_name, sdk_2.slug AS sdk_2_slug, count(DISTINCT app_sdk_1.app_id) AS count_1 
FROM app_sdk AS app_sdk_1, app_sdk AS app_sdk_2, sdk AS sdk_1, sdk AS sdk_2 
WHERE app_sdk_1.app_id = app_sdk_2.app_id AND app_sdk_1.sdk_id = sdk_1.id AND app_sdk_2.sdk_id = sdk_2.id AND app_sdk_1.installed = 0 AND app_sdk_2.installed = 1 AND sdk_1.slug IN (?, ?, ?) AND (sdk_2.slug NOT IN (?, ?, ?)) GROUP BY sdk_1.id
2022-09-11 20:00:18,793 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ('paypal', 'braintree', 'stripe', 'paypal', 'braintree', 'stripe')

paypal 139
stripe 50
braintree 51


In [8]:
totals = (
  session
  .query(func.count(distinct(AppSdk.app_id)), AppSdk, Sdk)
  .filter(
    AppSdk.sdk_id==Sdk.id,
    AppSdk.installed == True,
    Sdk.slug.in_(sdk_choice),
   )
  .group_by(Sdk.id)
).all()

print()
for count, _, sdk in totals:
  print(sdk, count)

2022-09-11 20:00:18,847 INFO sqlalchemy.engine.Engine SELECT count(DISTINCT app_sdk.app_id) AS count_1, app_sdk.app_id AS app_sdk_app_id, app_sdk.sdk_id AS app_sdk_sdk_id, app_sdk.installed AS app_sdk_installed, sdk.id AS sdk_id, sdk.name AS sdk_name, sdk.slug AS sdk_slug 
FROM app_sdk, sdk 
WHERE app_sdk.sdk_id = sdk.id AND app_sdk.installed = 1 AND sdk.slug IN (?, ?, ?) GROUP BY sdk.id
2022-09-11 20:00:18,851 INFO sqlalchemy.engine.Engine [generated in 0.00362s] ('paypal', 'braintree', 'stripe')

<SDK (name='PayPal' slug='paypal')> 84
<SDK (name='Stripe' slug='stripe')> 172
<SDK (name='Braintree' slug='braintree')> 145


In [9]:
not_involved = (
  session
  .query(func.count(App.id))
  .filter(
    ~session
    .query(AppSdk, Sdk)
    .filter(AppSdk.sdk_id == Sdk.id )
    .filter(Sdk.slug.in_(sdk_choice))
    .filter(AppSdk.installed == 1 )
    .filter(AppSdk.app_id == App.id)
    .exists()
  )
).all()


print()
for count, in not_involved:
  print(count)


2022-09-11 20:00:18,874 INFO sqlalchemy.engine.Engine SELECT count(app.id) AS count_1 
FROM app 
WHERE NOT (EXISTS (SELECT 1 
FROM app_sdk, sdk 
WHERE app_sdk.sdk_id = sdk.id AND sdk.slug IN (?, ?, ?) AND app_sdk.installed = ? AND app_sdk.app_id = app.id))
2022-09-11 20:00:18,875 INFO sqlalchemy.engine.Engine [generated in 0.00202s] ('paypal', 'braintree', 'stripe', 1)

9657
